# Table of Contents
* [Import data and Recover the 5-Fold Validation Indices](#Import-data-and-Recover-the-5-Fold-Validation-Indices)
* [Use n-gram to create feature matrix](#Use-n-gram-to-create-feature-matrix)
    * [Multinomial Naive Bayes](#Multinomial-Naive-Bayes)

In [2]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
import warnings
warnings.filterwarnings("ignore")

In [3]:
#import sys
#!conda install --yes --prefix {sys.prefix} -c conda-forge lightgbm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re
%matplotlib inline
# Training and Testing data files are available in the "../python notebook v1" directory.
# The following function finds all the .csv files in the project directory
import os
#def find_csv_filenames(path_to_dir, suffix=".csv"):
#    filenames = os.listdir(path_to_dir)
#    return [filename for filename in filenames if filename.endswith(suffix)]
#filenames = find_csv_filenames("../python notebook v1")
#for name in filenames:
#    print(name)

pd.options.display.max_colwidth = 200
import statistics
from scipy import stats
from scipy.stats import t
from scipy.stats import norm
import seaborn as sns
import sklearn
import sqlite3
from sqlite3 import Error
import csv
import lightgbm

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

## Import data and Recover the 5-Fold Validation Indices

In [4]:
X_train = pd.read_csv("../python notebook v1/X_train.csv", )
y_train_0 = pd.read_csv("../python notebook v1/y_train.csv", header=None)
y_train = y_train_0[0]
X_test = pd.read_csv("../python notebook v1/X_test.csv", )
y_test_0 = pd.read_csv("../python notebook v1/y_test.csv", header=None)
y_test = y_test_0[0]

In [5]:
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (11640, 9)
y_train shape:  (11640,)
X_test shape:  (2911, 9)
y_test shape:  (2911,)


In [6]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=123, shuffle=True)
kf.get_n_splits(X_train)

5

In [7]:
i = 1
for train_index, val_index in kf.split(X_train):
    print(f"Fold {i}: ", "Train:", train_index, "Validation:", val_index)
    i = i + 1

Fold 1:  Train: [    0     1     2 ... 11636 11638 11639] Validation: [    3     6     7 ... 11619 11620 11637]
Fold 2:  Train: [    1     3     5 ... 11637 11638 11639] Validation: [    0     2     4 ... 11591 11624 11629]
Fold 3:  Train: [    0     1     2 ... 11637 11638 11639] Validation: [    8    33    35 ... 11631 11635 11636]
Fold 4:  Train: [    0     2     3 ... 11636 11637 11638] Validation: [    1     5    18 ... 11632 11634 11639]
Fold 5:  Train: [    0     1     2 ... 11636 11637 11639] Validation: [   13    16    17 ... 11628 11633 11638]


## Use n-gram to create feature matrix

In [8]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# Create CountVectoroizer object
vectorizer = CountVectorizer(ngram_range=(1,2))
# Generate matrix of word vectors
X_train_matrix = vectorizer.fit_transform(X_train.text)
X_test_matrix = vectorizer.transform(X_test.text)

In [9]:
# Convert X_train_matrix into a DataFrame
X_train_df = pd.DataFrame(X_train_matrix.toarray())
# Map the column names to vocabulary 
X_train_df.columns = vectorizer.get_feature_names()
# keep the original ID column
X_train_df['ID'] = X_train.ID.to_list()
print(X_train_df.shape)
display(X_train_df.head())

(11640, 62226)


,aa,aa abi,aa able,aa account,aa advantage,aa agent,aa air,aa also,aa always,aa app,...,zoom sauce,zuke,zuke non,zurich,zurich bc,zurich jfk,zurich new,zurich would,zz,ID
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7908
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,538
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4501
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13982
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6048


In [10]:
print(X_train.text[1])
X_train_df.iloc[1,:][X_train_df.iloc[1,:] > 0]

submit response link send


link                 1
link send            1
response             1
response link        1
send                 1
submit               1
submit response      1
ID                 538
Name: 1, dtype: int64

In [11]:
X_train_leftover = X_train.drop(['text'], axis=1)
#display(X_train_leftover.head())
#display(X_train_df.head())
#print("Check the length are equal", len(X_train_leftover) == len(X_train_df))

# Get the full features by merging the 2 dataframes
X_train_final = pd.merge(X_train_df, X_train_leftover, how='inner', on='ID')
print(X_train_final.shape)
display(X_train_final.head())

(11640, 62233)


,aa,aa abi,aa able,aa account,aa advantage,aa agent,aa air,aa also,aa always,aa app,...,zurich would,zz,ID,retweet_count,airline_company_American,airline_company_Delta,airline_company_Southwest,airline_company_US Airways,airline_company_United,airline_company_Virgin America
0,0,0,0,0,0,0,0,0,0,0,...,0,0,7908,0.0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,538,0.0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,4501,0.0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,13982,0.0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,6048,0.0,0,0,1,0,0,0


In [12]:
# Drop the ID column to prepare for fitting models
X_train_no_ID = X_train_final.drop(['ID'], axis=1)
display(X_train_no_ID.head())

,aa,aa abi,aa able,aa account,aa advantage,aa agent,aa air,aa also,aa always,aa app,...,zurich new,zurich would,zz,retweet_count,airline_company_American,airline_company_Delta,airline_company_Southwest,airline_company_US Airways,airline_company_United,airline_company_Virgin America
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,1,0,0,0


### Multinomial Naive Bayes

Note: RandomForest, LightGBM and logistic regression models take very long to train. Given the limited compuatation power we have, in N-Gram case we only fit NB model. The training and validation scores indicate that there's some overfitting in the model.

In [13]:
# Import MultinomialNB
from sklearn.naive_bayes import MultinomialNB # Create MultinomialNB object
clf = MultinomialNB()

# Train clf on 5 fold data
val_accuracy = []
train_acc = []
for train_index, val_index in kf.split(X_train):
    X_train_5cv, X_val_5cv = X_train_no_ID.iloc[train_index,:], X_train_no_ID.iloc[val_index,:]
    y_train_5cv, y_val_5cv = y_train[train_index], y_train[val_index]
    clf.fit(X_train_5cv, y_train_5cv)
    train_acc.append(clf.score(X_train_5cv, y_train_5cv))
    val_accuracy.append(clf.score(X_val_5cv, y_val_5cv))
print("5-Fold Training scores are: ", train_acc)
mean_train_score = np.mean(train_acc)
print("Average Training score is: {:.5f}".format(mean_train_score))
print("5-Fold Validation scores are: ", val_accuracy)
mean_validation_score_NB = np.mean(val_accuracy)
print("Average validation score is: {:.5f}".format(mean_validation_score_NB))

# write average validation score to csv file
rlt_dict = {}
rlt_dict['Average_val_score'] = [mean_validation_score_NB]
rlt_df = pd.DataFrame.from_dict(rlt_dict)
filename = "NB_N_Gram_cv_rlt.csv"
rlt_df.to_csv(filename)

5-Fold Training scores are:  [0.9033505154639175, 0.9017396907216495, 0.904639175257732, 0.9033505154639175, 0.9059278350515464]
Average Training score is: 0.90380
5-Fold Validation scores are:  [0.7525773195876289, 0.7508591065292096, 0.7452749140893471, 0.7487113402061856, 0.7366838487972509]
Average validation score is: 0.74682


In [14]:
clf = MultinomialNB()
clf.fit(X_train_no_ID, y_train)

# Compute accuracy on entire training set
train_accuracy = clf.score(X_train_no_ID, y_train) 
print("Training accuracy: %f" % (train_accuracy))

# create confusion matrix and classification report for the training data
preds = clf.predict(X_train_no_ID)
confusion_matrix = sklearn.metrics.confusion_matrix(y_train, preds, labels=None, sample_weight=None)
report = classification_report(y_train, preds)
probs = clf.predict_proba(X_train_no_ID)
print("Confusion Matrix for training set is: \n", confusion_matrix)
print("Classification Report for training set is: \n", report)

Training accuracy: 0.912027
Confusion Matrix for training set is: 
 [[1820   45  576]
 [  35 1524  308]
 [  45   15 7272]]
Classification Report for training set is: 
               precision    recall  f1-score   support

           0       0.96      0.75      0.84      2441
           1       0.96      0.82      0.88      1867
           2       0.89      0.99      0.94      7332

   micro avg       0.91      0.91      0.91     11640
   macro avg       0.94      0.85      0.89     11640
weighted avg       0.92      0.91      0.91     11640

